In [ ]:
from image_acquisition.image_sensor import *
from image_acquisition.playback_images import *
from image_acquisition.video_file_writer import *
#from image_acquisition.ueye_python_driver import UEyePythonDriver

from feature_computation.object_tracking import *
from feature_computation.object_detection_and_tracking import *
from feature_computation.visualization import *

import matplotlib as plt

In [ ]:
# Load the ONNX model

output_path = "../../../Daten/makerspace_20241126/output/object_tracking_20241127.mp4"
raw_output_path =  "../../../Daten/makerspace_20241126/output/object_tracking_raw_20241127.mp4"


onnx_model_path = "../../Daten/makerspace_20241126/export/onnx/simplified-model.onnx"
print(os.path.realpath(onnx_model_path))
video_path = "../../Daten/makerspace_20241126/videos/soldering_recordings/Stefan/rec_1.avi"
print(os.path.realpath(video_path))

trackers = {
   "pad": MultiObjectTracker(class_ids=[0,1,4,5], disappearance_threshold=-1, ratio_threshold=0.7),
   "cabel": MultiObjectTracker(class_ids=2, disappearance_threshold=50, ratio_threshold=0.7),
   "soldering_iron": KalmanFilterTracker(class_ids=3, ratio_threshold=0.7, disappearance_threshold=70)
}

In [ ]:

visualizer = ObjectTrackerVisualization(trackers)

In [ ]:
import state_update as su

pads = [su.pad(), su.pad()]

In [ ]:
def float_state_to_string_state(float_state):
    if float_state == 0.0:
        return "presoldered"
    if float_state == 1.0:
        return "soldered"
    if float_state == 4.0:
        return "unsoldered"
    if float_state == 5.0:
        return "in_process"
    raise Exception("unknown pad state")


try:
    with VideoFileImageSensor(video_path=video_path) as img_src:
    #with UEyePythonDriver() as img_src:
        with video_file_writer_matching_image_sensor(img_src, output_path) as writer:
            with onnx_network_and_trackers(onnx_model_path, trackers.values()) as network_and_trackers:

                def callback(frame:np.ndarray):
                    network_and_trackers.update_trackers(frame)
                    disp_img = frame.copy()
                    visualizer.draw_visualization_on_frame(disp_img)


                    height, width = disp_img.shape[:2]
                    new_width = width // 3
                    new_height = height // 3

                    # Resize the image
                    disp_img = cv2.resize(disp_img, (new_width, new_height), interpolation=cv2.INTER_AREA)
                    cv2.imshow("object tracking", disp_img)

                    writer.write(disp_img)

                    #update the global variables in state_update with data from object detection
                    su.iron_pos = None

                    for si in trackers["soldering_iron"].get_objects():
                        su.iron_pos = si.bounding_box
                    
                    su.wire_pos = []
                    for w in trackers["cabel"].get_objects():
                        su.wire_pos.append(w.bounding_box)                        

                    detected_pads = trackers["pad"].get_objects()
                    
                    for dp in detected_pads:
                        pads[dp.id].pad_pos = dp.bounding_box#
                        pads[dp.id].pad_state = float_state_to_string_state(dp.state)


                    for i, p in enumerate(pads):
                        
                        print("state of pad ", str(i), ": ", p.state_update())



                    # Check if 'q' is pressed to quit
                    if cv2.waitKey(1) & 0xFF == ord('q'):
                        raise NoMoreImages


                run_playback(img_src, callback)
finally:
    cv2.destroyAllWindows()